In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, '../')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np

#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars, computeData, createFakeDataset
import yaml

In [135]:
with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
N = params['number_of_cameras']

df_save = loadExcel('input.xlsx', N=params['number_of_cameras'])

In [136]:
df = df_save.copy(deep=True)
selectedDirections = [1,3,10,12]

In [146]:
writer.close()
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
    
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetTimes(df.copy(True), writer, params)
    writer.close()


In [100]:
writer.close()

In [118]:
combinedIndexesString = "_".join([f'{i}' for i in selectedDirections])

def checkForCorrectDirections(x):
    values = [str(x[f'Direction_{i}']) for i in range(len(selectedDirections))]
    return combinedIndexesString == "_".join(values)


temp = df_save.copy(deep=True)
temp = temp[temp['License_plate'] != 'unknown']

# filter time
temp = temp.set_index('Capture_time').between_time(startTime, endTime).reset_index()

# add fake data to assure that everything goes smoothly
fake_data = {'License_plate': [], 'Capture_time': [], 'Direction': []}
for i, dire in enumerate(selectedDirections):
    fake_data['License_plate'].append('FakeSPZ')
    fake_data['Capture_time'].append(pd.to_datetime(f'00:{i:02d}'))
    fake_data['Direction'].append(dire)
fake_df = pd.DataFrame().from_dict(fake_data)
temp = temp.append(fake_df, ignore_index=True)

# create wide format
temp = temp.sort_values(['License_plate', 'Capture_time'], ascending=True).reset_index(drop=True)
temp['index'] = temp.index
temp = temp.rename(
    columns={"Capture_time": f"Capture_time_{0}", "Direction": f"Direction_{0}"})

temp_save = temp

for i in range(1, len(selectedDirections)):
    temp_moved = temp_save.copy(deep=True)
    temp_moved['index'] = temp_moved.index - i
    temp_moved = temp_moved.rename(
        columns={"Capture_time_0": f"Capture_time_{i}", "Direction_0": f"Direction_{i}"})

    temp = temp.merge(temp_moved, how='inner', on=['index', 'License_plate'])

    
temp = temp.drop(columns=['index'])
temp = temp[temp.agg(checkForCorrectDirections,axis=1)]

# remove fake data
temp = temp[temp['License_plate'] != 'FakeSPZ']

# add time difference
for i in range(1, len(selectedDirections)):
    temp[f'time_{i - 1}'] = temp[f'Capture_time_{i}'] - temp[f'Capture_time_{i - 1}']

# remove unused columns
for i in range(0, len(selectedDirections)):
    temp = temp.drop(columns=[f'Direction_{i}'])

# convert time columns to better number
for i in range(0, len(selectedDirections) - 1):
    temp[f'time_{i}'] = temp[f'time_{i}'].dt.seconds / 60


In [119]:
temp

,License_plate,Capture_time_0,Capture_time_1,Capture_time_2,Capture_time_3,time_0,time_1,time_2
37,1A85174,2021-10-07 16:37:33,2021-10-07 16:39:02,2021-10-07 16:44:30,2021-10-07 16:47:53,1.483333,5.466667,3.383333
91,1AD5063,2021-10-07 18:15:05,2021-10-07 18:16:38,2021-10-07 18:25:35,2021-10-07 18:26:54,1.550000,8.950000,1.316667
159,1AJ6182,2021-10-07 17:24:23,2021-10-07 17:26:08,2021-10-07 18:48:22,2021-10-07 18:49:46,1.750000,82.233333,1.400000
169,1AK3888,2021-10-07 17:25:43,2021-10-07 17:27:06,2021-10-07 17:38:03,2021-10-07 17:39:55,1.383333,10.950000,1.866667
215,1AM0720,2021-10-07 16:15:56,2021-10-07 16:17:42,2021-10-07 16:27:03,2021-10-07 16:29:16,1.766667,9.350000,2.216667
...,...,...,...,...,...,...,...,...
11287,BOL6695,2021-10-07 15:53:56,2021-10-07 15:55:27,2021-10-07 16:00:52,2021-10-07 16:03:14,1.516667,5.416667,2.366667
11334,EL473AP,2021-10-07 15:55:08,2021-10-07 15:56:27,2021-10-07 16:02:02,2021-10-07 16:04:43,1.316667,5.583333,2.683333
11376,EL731AC,2021-10-07 15:57:37,2021-10-07 16:11:28,2021-10-07 16:31:26,2021-10-07 16:34:36,13.850000,19.966667,3.166667
11443,KK749BN,2021-10-07 15:56:11,2021-10-07 15:57:38,2021-10-07 16:09:46,2021-10-07 16:12:20,1.450000,12.133333,2.566667
